In [ ]:
import pandas as pd
import numpy as np
import re
import datetime as time
from mlxtend.frequent_patterns import apriori, association_rules
import sys
np.set_printoptions(threshold=sys.maxsize)
pd.set_option('display.max_columns', None)
df = pd.read_parquet('scontrini_2.parquet',engine='pyarrow')
df = df[(~df.reparto.isin(["95","80","85","03"]))&(df.tipo=="")]
df

In [ ]:
def hot_encode(x):
    if(x==0):
        return False
    else:
        return True

Divisione di tutti i mesi

In [ ]:
settembre=df[(df.data_reg>=time.date(2021,9,1))&(df.data_reg<=time.date(2021,9,30))]
ottobre=df[(df.data_reg>=time.date(2021,10,1))&(df.data_reg<=time.date(2021,10,31))]
novembre=df[(df.data_reg>=time.date(2021,11,1))&(df.data_reg<=time.date(2021,11,30))]
dicembre=df[(df.data_reg>=time.date(2021,12,1))&(df.data_reg<=time.date(2021,12,31))]

In [ ]:
giorniScontiSet=settembre[settembre.promoNoPromo==1].groupby(['descrizione','data_reg'])["qta_riga"].sum().to_frame().reset_index()
giorniScontiOtt=ottobre[ottobre.promoNoPromo==1].groupby(['descrizione','data_reg'])["qta_riga"].sum().to_frame().reset_index()
giorniScontiNov=novembre[novembre.promoNoPromo==1].groupby(['descrizione','data_reg'])["qta_riga"].sum().to_frame().reset_index()
giorniScontiDic=dicembre[dicembre.promoNoPromo==1].groupby(['descrizione','data_reg'])["qta_riga"].sum().to_frame().reset_index()

In [ ]:
basketAux=settembre[settembre.promoNoPromo==1][['cassa','transazione','data_reg','descrizione','qta_riga']]\
    .groupby(['cassa','transazione','data_reg','descrizione'])['qta_riga']\
    .sum()\
    .unstack(fill_value=0)\
    .reset_index()\
    .drop(['cassa', 'transazione','data_reg'], axis=1)\
    .applymap(hot_encode)
mostSoldSet = list(apriori(basketAux, min_support = 0.03, use_colnames = True).sort_values(by=['support'], ascending=False)['itemsets'].iloc[0])[0]

basketAux=ottobre[ottobre.promoNoPromo==1][['cassa','transazione','data_reg','descrizione','qta_riga']]\
    .groupby(['cassa','transazione','data_reg','descrizione'])['qta_riga']\
    .sum()\
    .unstack(fill_value=0)\
    .reset_index()\
    .drop(['cassa', 'transazione','data_reg'], axis=1)\
    .applymap(hot_encode)
mostSoldOtt = list(apriori(basketAux, min_support = 0.02, use_colnames = True).sort_values(by=['support'], ascending=False)['itemsets'].iloc[0])[0]

basketAux=novembre[novembre.promoNoPromo==1][['cassa','transazione','data_reg','descrizione','qta_riga']]\
    .groupby(['cassa','transazione','data_reg','descrizione'])['qta_riga']\
    .sum()\
    .unstack(fill_value=0)\
    .reset_index()\
    .drop(['cassa', 'transazione','data_reg'], axis=1)\
    .applymap(hot_encode)
mostSoldNov = list(apriori(basketAux, min_support = 0.03, use_colnames = True).sort_values(by=['support'], ascending=False)['itemsets'].iloc[0])[0]

basketAux=dicembre[dicembre.promoNoPromo==1][['cassa','transazione','data_reg','descrizione','qta_riga']]\
    .groupby(['cassa','transazione','data_reg','descrizione'])['qta_riga']\
    .sum()\
    .unstack(fill_value=0)\
    .reset_index()\
    .drop(['cassa', 'transazione','data_reg'], axis=1)\
    .applymap(hot_encode)
mostSoldDic = list(apriori(basketAux, min_support = 0.03, use_colnames = True).sort_values(by=['support'], ascending=False)['itemsets'].iloc[0])[0]

In [ ]:
#settembre
inizioSet = giorniScontiSet[giorniScontiSet.descrizione==mostSoldSet].sort_values(by='data_reg')['data_reg'].iloc[0]
fineSet = giorniScontiSet[giorniScontiSet.descrizione==mostSoldSet].sort_values(by='data_reg')['data_reg'].iloc[-1]
i=-2
while(fineSet-inizioSet >= time.timedelta(14)):
    fineSet = giorniScontiSet[giorniScontiSet.descrizione==mostSoldSet].sort_values(by='data_reg')['data_reg'].iloc[i]
    i-=1

#ottobre
inizioOtt = giorniScontiOtt[giorniScontiOtt.descrizione==mostSoldOtt].sort_values(by='data_reg')['data_reg'].iloc[0]
fineOtt = giorniScontiOtt[giorniScontiOtt.descrizione==mostSoldOtt].sort_values(by='data_reg')['data_reg'].iloc[-1]
i=-2
while(fineOtt-inizioOtt >= time.timedelta(14)):
    fineOtt = giorniScontiOtt[giorniScontiOtt.descrizione==mostSoldOtt].sort_values(by='data_reg')['data_reg'].iloc[i]
    i-=1

#novembre
inizioNov = giorniScontiNov[giorniScontiNov.descrizione==mostSoldNov].sort_values(by='data_reg')['data_reg'].iloc[0]
fineNov = giorniScontiNov[giorniScontiNov.descrizione==mostSoldNov].sort_values(by='data_reg')['data_reg'].iloc[-1]
i=-2
while(fineNov-inizioNov >= time.timedelta(14)):
    fineNov = giorniScontiNov[giorniScontiNov.descrizione==mostSoldNov].sort_values(by='data_reg')['data_reg'].iloc[i]
    i-=1

#dicembre 
inizioDic = giorniScontiDic[giorniScontiDic.descrizione==mostSoldDic].sort_values(by='data_reg')['data_reg'].iloc[0]
fineDic = giorniScontiDic[giorniScontiDic.descrizione==mostSoldDic].sort_values(by='data_reg')['data_reg'].iloc[-1]
i=-2
while(fineDic-inizioDic >= time.timedelta(14)):
    fineDic = giorniScontiDic[giorniScontiDic.descrizione==mostSoldDic].sort_values(by='data_reg')['data_reg'].iloc[i]
    i-=1

In [ ]:
basketScontoSet=df[(df.data_reg>=inizioSet)&(df.data_reg<=fineSet)][['cassa','transazione','data_reg','descrizione','qta_riga']]\
    .groupby(['cassa','transazione','data_reg','descrizione'])['qta_riga']\
    .sum()\
    .unstack(fill_value=0)\
    .reset_index()\
    .fillna(0)\
    .drop(['cassa', 'transazione','data_reg'], axis=1)\
    .applymap(hot_encode)

basketScontoOtt=df[(df.data_reg>=inizioOtt)&(df.data_reg<=fineOtt)][['cassa','transazione','data_reg','descrizione','qta_riga']]\
    .groupby(['cassa','transazione','data_reg','descrizione'])['qta_riga']\
    .sum()\
    .unstack(fill_value=0)\
    .reset_index()\
    .drop(['cassa', 'transazione','data_reg'], axis=1)\
    .applymap(hot_encode)

basketScontoNov=df[(df.data_reg>=inizioNov)&(df.data_reg<=fineNov)][['cassa','transazione','data_reg','descrizione','qta_riga']]\
    .groupby(['cassa','transazione','data_reg','descrizione'])['qta_riga']\
    .sum()\
    .unstack(fill_value=0)\
    .reset_index()\
    .drop(['cassa', 'transazione','data_reg'], axis=1)\
    .applymap(hot_encode)

basketScontoDic=df[(df.data_reg>=inizioDic)&(df.data_reg<=fineDic)][['cassa','transazione','data_reg','descrizione','qta_riga']]\
    .groupby(['cassa','transazione','data_reg','descrizione'])['qta_riga']\
    .sum()\
    .unstack(fill_value=0)\
    .reset_index()\
    .drop(['cassa', 'transazione','data_reg'], axis=1)\
    .applymap(hot_encode)

In [ ]:
print("il prodotto scontato più venduto mese di settembre: ",mostSoldSet)

In [ ]:
frq_items_sconto_set = apriori(basketScontoSet, min_support = 0.001, use_colnames = True)
rules_sconto_set = association_rules(frq_items_sconto_set, metric ="confidence", min_threshold = 0.01).sort_values(by=['confidence'], ascending=False)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(rules_sconto_set)

Itemsets con il prodotto più venduto

In [ ]:
rules_sconto_set[(rules_sconto_set['antecedents'].astype(str).str.contains(mostSoldSet,regex=False))|
                ((rules_sconto_set['consequents'].astype(str).str.contains(mostSoldSet,regex=False)))]

In [ ]:
print("il prodotto scontato più venduto mese di ottobre: ",mostSoldOtt)

In [ ]:
frq_items_sconto_ott = apriori(basketScontoOtt, min_support = 0.002, use_colnames = True)
rules_sconto_ott = association_rules(frq_items_sconto_ott, metric ="confidence", min_threshold = 0.01).sort_values(by=['confidence'], ascending=False)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(rules_sconto_ott)

Itemsets con il prodotto più venduto

In [ ]:
rules_sconto_ott[(rules_sconto_ott['antecedents'].astype(str).str.contains(mostSoldOtt,regex=False))|
                ((rules_sconto_ott['consequents'].astype(str).str.contains(mostSoldOtt,regex=False)))]

In [ ]:
print("il prodotto scontato più venduto mese di novembre: ",mostSoldNov)

In [ ]:
frq_items_sconto_nov = apriori(basketScontoNov, min_support = 0.001, use_colnames = True)
rules_sconto_nov = association_rules(frq_items_sconto_nov, metric ="confidence", min_threshold = 0.01).sort_values(by=['confidence'], ascending=False)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(rules_sconto_nov)

Itemsets con il prodotto più venduto

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(rules_sconto_nov[(rules_sconto_nov['antecedents'].astype(str).str.contains(mostSoldNov,regex=False))|
                ((rules_sconto_nov['consequents'].astype(str).str.contains(mostSoldNov,regex=False)))])



In [ ]:
print("il prodotto scontato più venduto mese di dicembre: ",mostSoldDic)

In [ ]:
frq_items_sconto_dic = apriori(basketScontoDic, min_support = 0.002, use_colnames = True)
rules_sconto_dic = association_rules(frq_items_sconto_dic, metric ="confidence", min_threshold = 0.01).sort_values(by=['confidence'], ascending=False)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(rules_sconto_dic)

Itemsets con il prodotto più venduto

In [ ]:
rules_sconto_dic[(rules_sconto_dic['antecedents'].astype(str).str.contains(mostSoldDic,regex=False))|
                ((rules_sconto_dic['consequents'].astype(str).str.contains(mostSoldDic,regex=False)))]